In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) 
from os import listdir
import os
import glob
import spacy
from collections import Counter
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from spacy.scorer import Scorer
import re
import nlp

[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-11-11 18:53:37.851674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#load data
job_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data"
courses_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/courses_data"
skills_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/Datasets/skills_df_updated.csv"
data_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data"
files_j= [f for f in listdir(job_path) if f.endswith(".csv")]
df_j = pd.concat([pd.read_csv(os.path.join(job_path,f)) for f in files_j])

files_c= [f for f in listdir(courses_path) if f.endswith(".csv")]
df_c = pd.concat([pd.read_csv(os.path.join(courses_path,f)) for f in files_c])

#files_s= [f for f in listdir(skills_path) if f.endswith(".csv")]
df_s = pd.read_csv(skills_path)



In [128]:


# Ensure you have the necessary NLTK tokens downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def preprocess_text_spacy(text):
    # Use spaCy's pipeline for processing the passed text
    doc = nlp(text)
    preprocessed_text= []
    # Return a preprocessed version of the text (for instance, lemmatized, lowercased, no stop words/punctuations)
    for token in doc:
        # Skip stop words and punctuation
        if token.is_stop or token.is_punct or token.is_space:
            continue
        # Special case for 'data' to keep it in plural form
        if token.lemma_ == 'datum':
            preprocessed_text.append('data')
        else:
            preprocessed_text.append(token.text.lower())

    return ' '.join(preprocessed_text)


# Function to clean and lemmatize job title text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = nltk.word_tokenize(text)
    #words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

def clean_lem_stop(df,column_name):
    df[column_name] = df[column_name].apply(preprocess_text_spacy)
    df[column_name] = df[column_name].apply(clean_text)
    return df





[nltk_data] Downloading package punkt to /Users/nyzy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nyzy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [179]:
#clean skills dataset 

skills = pd.DataFrame(df_s[df_s["Unnamed: 5"].notna()]["Unnamed: 5"].unique())
skills.columns=["skills"]
#skills["skills"] = skills["skills"].apply(preprocess_text_spacy)
skills["skills"] = skills["skills"].apply(clean_text)
skills_list = skills["skills"].tolist()
skills_list.append('python')
skills_list.append('python programming')
skills_list.append('statistical')
skills_list.append("r programming")


['visualization',
 'b testing',
 'ansi c',
 'api gateway',
 'api management',
 'apl programming language',
 'asp net',
 'asp net core',
 'asp net core mvc',
 'asp net extension ajax',
 'asp net fundamental',
 'asp net identity',
 'asp net mvc',
 'asp net mvc framework',
 'asp net razor',
 'asp net web api',
 'awk programming language',
 'aws amplify',
 'aws app mesh',
 'aws appsync',
 'aws auto scaling',
 'aws backup',
 'aws batch',
 'aws cli command line interface',
 'aws cloud development kit cdk',
 'aws cloudformation',
 'aws cloudhsm',
 'aws cloudtrail',
 'aws codebuild',
 'aws codecommit',
 'aws codedeploy',
 'aws codepipeline',
 'aws cost management',
 'aws directory service',
 'aws elastic beanstalk',
 'aws elastic mapreduce emr',
 'aws fargate',
 'aws glue',
 'aws identity access management iam',
 'aws inferentia',
 'aws internet thing iot',
 'aws key management service km',
 'aws kinesis',
 'aws lambda',
 'aws opsworks',
 'aws outpost',
 'aws sdk',
 'aws sagemaker',
 'aws serv

In [15]:
df_j = df_j[:100]
df_c = df_c[:100]

In [189]:
def clean_job_df(df):
    df = clean_lem_stop(df,"title")
    df = clean_lem_stop(df,"jobDescription")
    job_descriptions = df_j["jobDescription"].tolist()
    return df,job_descriptions

df_j_cleaned,job_descriptions = clean_job_df(df_j)

columns_of_interest = ['company',
 'companyRating',
 'companyReviewCount',
 'displayTitle',
 'employerAssistEnabled',
 'employerResponsive',
 'extractedSalary',
 'featuredEmployer',
 'featuredEmployerCandidate',
 'formattedLocation',
 'formattedRelativeTime',
 'highVolumeHiringModel',
 'hiringEventJob',
 'indeedApplyEnabled',
 'indeedApplyable',
 'isJobVisited',
 'isMobileThirdPartyApplyable',
 'isNoResumeJob',
 'isSubsidiaryJob',
 'jobCardRequirementsModel',
 'jobLocationCity',
 'jobLocationState',
 'locationCount',
 'newJob',
 'normTitle',
 'openInterviewsInterviewsOnTheSpot',
 'openInterviewsJob',
 'openInterviewsOffersOnTheSpot',
 'openInterviewsPhoneJob',
 'overrideIndeedApplyText',
 'remoteLocation',
 'resumeMatch',
 'salarySnippet',
 'showAttainabilityBadge',
 'showCommutePromo',
 'showEarlyApply',
 'showJobType',
 'showRelativeDate',
 'showSponsoredLabel',
 'showStrongerAppliedLabel',
 'smartFillEnabled',
 'smbD2iEnabled',
 'snippet',
 'sponsored',
 'title',
 'truncatedCompany',
 'urgentlyHiring',
 'vjFeaturedEmployerCandidate',
 'jobDescription']


import ast 
import numpy as np
def salary_to_dict(row):
    if pd.isna(row):
        # Return a dictionary with NaN values if the row is NaN
        return {'max': np.nan, 'min': np.nan, 'type': np.nan}
    elif isinstance(row, dict):
        # Return the row as is if it's already a dictionary
        return row
    else:
        # If the row is a string representation of a dictionary (assumed if not NaN or dict),
        # safely evaluate it to a dictionary here
        # Note: Be cautious with `eval`. Here it's mentioned for potential string to dict conversion.
        # In a secure context, confirm the string format and consider `ast.literal_eval` instead.
        try:
            # Convert string to dictionary safely
            dict_row = eval(row)
            return dict_row if isinstance(dict_row, dict) else {'max': np.nan, 'min': np.nan, 'type': np.nan}
        except:
            # In case of error during eval, return NaN values
            return {'max': np.nan, 'min': np.nan, 'type': np.nan}

# Apply the function to each row of the 'extractedSalary' column
salary_dicts = df_j_cleaned['extractedSalary'].apply(salary_to_dict)
# Now that we have a series of dictionaries, use `json_normalize` to create a DataFrame
salary_df = pd.json_normalize(salary_dicts)

# Concatenate the new DataFrame with the original one
df_j_cleaned = pd.concat([df_j_cleaned.drop('extractedSalary', axis=1).reset_index(), salary_df], axis=1)

#df["extractedSalary"]=df["extractedSalary"].astype(str)
#df["extractedSalary"]=df['extractedSalary'].apply(ast.literal_eval)

import ast 
import numpy as np
def salary_snippet_to_dict(row):
    if pd.isna(row):
        # Return a dictionary with NaN values if the row is NaN
        return {'currency': np.nan, 'salaryTextFormatted': np.nan, 'source': np.nan,'text':np.nan}
    elif isinstance(row, dict):
        # Return the row as is if it's already a dictionary
        return row
    else:
        # If the row is a string representation of a dictionary (assumed if not NaN or dict),
        # safely evaluate it to a dictionary here
        # Note: Be cautious with `eval`. Here it's mentioned for potential string to dict conversion.
        # In a secure context, confirm the string format and consider `ast.literal_eval` instead.
        try:
            # Convert string to dictionary safely
            dict_row = eval(row)
            return dict_row if isinstance(dict_row, dict) else {'max': np.nan, 'min': np.nan, 'type': np.nan}
        except:
            # In case of error during eval, return NaN values
            return {'currency': np.nan, 'salaryTextFormatted': np.nan, 'source': np.nan,'text':np.nan}

# Apply the function to each row of the 'extractedSalary' column
salary_snippet_dicts = df_j_cleaned['salarySnippet'].apply(salary_snippet_to_dict)
# Now that we have a series of dictionaries, use `json_normalize` to create a DataFrame
salary_snippet_df = pd.json_normalize(salary_snippet_dicts)

# Concatenate the new DataFrame with the original one
df_j_cleaned = pd.concat([df_j_cleaned.drop('salarySnippet', axis=1).reset_index(), salary_snippet_df], axis=1)

#df["extractedSalary"]=df["extractedSalary"].astype(str)
#df["extractedSalary"]=df['extractedSalary'].apply(ast.literal_eval)


In [190]:

#phrase matcher model for skills

import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.util import filter_spans

def phrase_matcher_model(description,skills_list):
    nlp = spacy.load("en_core_web_md")  # Load the model
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")  # Create the matcher object

    # Assuming 'skills_list' is a list of skills, and 'job_descriptions' is a list containing job descriptions

    # Add patterns to the matcher. Patterns are made by converting each skill string into a Doc object
    patterns = [nlp.make_doc(skill) for skill in skills_list]
    matcher.add("Skills", patterns)

    # Process the job description to create a Spacy Doc
    doc = nlp(description)

    # Match the patterns to the doc
    matches = matcher(doc)

    # Create Span objects for the matched sequences
    spans = [Span(doc, start, end, label="SKILL") for match_id, start, end in matches]

    # Filter the spans to remove overlaps
    filtered_spans = filter_spans(spans)

    # Now you can create new entities in the doc using the filtered spans
    doc.ents = filtered_spans  # Overwrite or append to doc.ents with the non-overlapping skill entities
    entities_extracted = []
    # Print the entities in the document
    for ent in doc.ents:
        entities_extracted.append(ent.text)
    '''
    for each in matches:
        #print(each)
        if each.lower_ == "statistical":
            for skills_identify in skills_list:
                if each.lower_ in skills_identify:
                    #print(skills_identify)
                    entities_extracted.append(skills_identify)
    '''
    return set(entities_extracted)

df_j_cleaned["skills_tagged"]=df_j_cleaned["jobDescription"].apply(lambda x:phrase_matcher_model(x,skills_list))




In [191]:

#job progression dictionary
# Define the original dictionary provided by the user

Job_progression_dictionary =  {
"Junior Data Analyst" : ["Data Analyst", "Senior Data Analyst", "Data Engineer", "Senior Data Engineer", "Lead Data Engineer", "Data Architect", "Data Manager", "Director of Analytics", "Chief Data Officer"],
"Junior Data Scientist" : ["Data Scientist", "Senior Data Scientist", "Lead Data Scientist", "Principal Data Scientist", "Data Science Manager", "Director of Data Science", "Vice President of Data Science", "Chief Data Scientist"],
"Junior Data Engineer" : ["Data Engineer", "Senior Data Engineer", "Data Pipeline Architect", "Data Engineering Manager", "Director of Data Engineering", "Chief Technology Officer (CTO)"],
"Business Intelligence Analyst" : ["Senior Business Intelligence Analyst", "Business Intelligence Manager", "Director of Business Intelligence", "Vice President of Business Intelligence", "Chief Intelligence Officer"],
"Machine Learning Engineer" : ["Senior Machine Learning Engineer", "Machine Learning Architect", "Machine Learning Manager", "Head of Machine Learning", "Chief AI Officer"],
"Data Analyst" : ["Data Engineer", "Senior Data Engineer", "Data Architect", "Senior Data Architect", "Enterprise Architect", "Chief Technology Officer (CTO)"],
"Data Analyst" : ["Big Data Engineer", "Senior Big Data Engineer", "Big Data Architect", "Head of Big Data", "Director of Data Engineering", "Chief Information Officer (CIO)"],
"Junior AI Developer" : ["AI Developer", "Senior AI Developer", "AI Architect", "AI Project Manager", "Head of AI", "Chief AI Officer"],
"Junior DBA" : ["Mid-level DBA", "Senior DBA", "Database Manager", "Data Architect", "Director of Database Management", "Chief Information Officer (CIO)"],
"Junior Statistician" : ["Statistician" , "Senior Statistician", "Quantitative Analyst", "Senior Quantitative Analyst", "Quantitative Research Manager", "Director of Quantitative Research"],
"Junior Business Analyst" : ["Business Analyst", "Senior Business Analyst", "Business Analysis Manager", "Business Intelligence Analyst", "Director of Business Analysis", "Chief Strategy Officer"],
"Research Assistant" : ["Research Analyst", "Research Scientist", "Senior Research Scientist", "Principal Scientist", "Director of Research", "Chief Science Officer"],
"Data Analyst" : ["Product Analyst", "Data Product Manager", "Senior Data Product Manager", "Director of Product Management", "Vice President of Product", "Chief Product Officer"]
}

original_dict = {
    "Junior Data Analyst": ["Data Analyst", "Senior Data Analyst", "Data Engineer", "Senior Data Engineer", 
                            "Lead Data Engineer", "Data Architect", "Data Manager", "Director of Analytics", 
                            "Chief Data Officer"],
    "Junior Data Scientist": ["Data Scientist", "Senior Data Scientist", "Lead Data Scientist", 
                              "Principal Data Scientist", "Data Science Manager", "Director of Data Science", 
                              "Vice President of Data Science", "Chief Data Scientist"],
    "Junior Data Engineer": ["Data Engineer", "Senior Data Engineer", "Data Pipeline Architect", 
                             "Data Engineering Manager", "Director of Data Engineering", "Chief Technology Officer (CTO)"],
    "Business Intelligence Analyst": ["Senior Business Intelligence Analyst", "Business Intelligence Manager", 
                                      "Director of Business Intelligence", "Vice President of Business Intelligence", 
                                      "Chief Intelligence Officer"],
    "Machine Learning Engineer": ["Senior Machine Learning Engineer", "Machine Learning Architect", 
                                  "Machine Learning Manager", "Head of Machine Learning", "Chief AI Officer"],
    "Junior AI Developer": ["AI Developer", "Senior AI Developer", "AI Architect", "AI Project Manager", 
                            "Head of AI", "Chief AI Officer"],
    "Junior DBA": ["Mid-level DBA", "Senior DBA", "Database Manager", "Data Architect", "Director of Database Management", 
                   "Chief Information Officer (CIO)"],
    "Junior Statistician": ["Statistician", "Senior Statistician", "Quantitative Analyst", "Senior Quantitative Analyst", 
                            "Quantitative Research Manager", "Director of Quantitative Research"],
    "Junior Business Analyst": ["Business Analyst", "Senior Business Analyst", "Business Analysis Manager", 
                                "Business Intelligence Analyst", "Director of Business Analysis", "Chief Strategy Officer"],
    "Research Assistant": ["Research Analyst", "Research Scientist", "Senior Research Scientist", "Principal Scientist", 
                           "Director of Research", "Chief Science Officer"],
    # Multiple entries for Data Analyst have been combined to include all unique progressions
    "Data Analyst": ["Data Engineer", "Senior Data Engineer", "Lead Data Engineer", "Data Architect", 
                     "Senior Data Architect", "Enterprise Architect", "Product Analyst", "Data Product Manager", 
                     "Senior Data Product Manager", "Director of Product Management", "Vice President of Product", 
                     "Chief Product Officer", "Big Data Engineer", "Senior Big Data Engineer", "Big Data Architect", 
                     "Head of Big Data", "Director of Data Engineering", "Chief Information Officer (CIO)"]
}

# Define lateral moves for the given roles
lateral_moves = {
    "Data Analyst": ["Business Intelligence Analyst", "Machine Learning Engineer"],
    "Data Scientist": ["Data Engineer", "AI Developer"],
    "Data Engineer": ["Machine Learning Engineer", "Big Data Engineer"],
    "Business Intelligence Analyst": ["Data Analyst", "Data Scientist"],
    "Machine Learning Engineer": ["Data Scientist", "AI Developer"],
    "AI Developer": ["Machine Learning Engineer", "Data Engineer"],
    "DBA": ["Data Engineer", "Data Analyst"],
    "Statistician": ["Data Analyst", "Data Scientist"],
    "Business Analyst": ["Data Analyst", "Business Intelligence Analyst"],
    "Research Analyst": ["Data Scientist", "Statistician"]
}

# Since we want to include lateral moves for each value in the original dictionary, 
# we will create a function that merges the direct progressions and lateral moves into one list.

# Function to merge progression and lateral moves
def merge_progression_and_lateral_moves(direct_progression, lateral_move_titles):
    # Start with direct progression
    full_progression = direct_progression.copy()
    
    # Add lateral moves for each title in the direct progression if they exist
    for title in direct_progression:
        lateral_titles = lateral_moves.get(title, [])
        for lateral_title in lateral_titles:
            if lateral_title not in full_progression:  # Avoid duplicates
                full_progression.append(lateral_title)
    
    return full_progression

# Function to build a full job progression dictionary for each title
def build_full_progression_dict(original_dict, lateral_moves):
    full_progression_dict = {}
    
    # Iterate over each starting job title
    for start_title, progression in original_dict.items():
        # Get the full progression for the starting title
        full_progression = merge_progression_and_lateral_moves(progression, lateral_moves)
        
        # Add the full progression to the dictionary for the starting title
        full_progression_dict[start_title] = full_progression
        
        # Now iterate over each job within the progression to build their own progression paths
        for i, title in enumerate(progression):
            if title not in full_progression_dict:  # Only add if it doesn't already exist to avoid overwriting
                # Get the progression for this title (which is the rest of the list after this title)
                next_progression = merge_progression_and_lateral_moves(progression[i + 1:], lateral_moves)
                full_progression_dict[title] = next_progression
    
    return full_progression_dict

# Building the full job progression dictionary
full_job_progression_dict = build_full_progression_dict(original_dict, lateral_moves)

# Sorting the dictionary for better readability
sorted_full_job_progression_dict = {k: full_job_progression_dict[k] for k in sorted(full_job_progression_dict)}


unique_job_title_full = set()
for key,value in Job_progression_dictionary.items():
    unique_job_title_full.add(key)
    for each in value:
        unique_job_title_full.add(each)


unique_job_title = set()
for key,value in sorted_full_job_progression_dict.items():
    unique_job_title.add(key)
    for each in value:
        unique_job_title.add(each)

unique_job_title.difference(unique_job_title_full)

sorted_full_job_progression_dict



{'AI Architect': ['AI Project Manager', 'Head of AI', 'Chief AI Officer'],
 'AI Developer': ['Senior AI Developer',
  'AI Architect',
  'AI Project Manager',
  'Head of AI',
  'Chief AI Officer'],
 'AI Project Manager': ['Head of AI', 'Chief AI Officer'],
 'Big Data Architect': ['Head of Big Data',
  'Director of Data Engineering',
  'Chief Information Officer (CIO)'],
 'Big Data Engineer': ['Senior Big Data Engineer',
  'Big Data Architect',
  'Head of Big Data',
  'Director of Data Engineering',
  'Chief Information Officer (CIO)'],
 'Business Analysis Manager': ['Business Intelligence Analyst',
  'Director of Business Analysis',
  'Chief Strategy Officer',
  'Data Analyst',
  'Data Scientist'],
 'Business Analyst': ['Senior Business Analyst',
  'Business Analysis Manager',
  'Business Intelligence Analyst',
  'Director of Business Analysis',
  'Chief Strategy Officer',
  'Data Analyst',
  'Data Scientist'],
 'Business Intelligence Analyst': ['Senior Business Intelligence Analyst',
 

In [192]:
#clean title
import re
import numpy as np
from fuzzywuzzy import process
#base_job_titles = list(sorted_full_job_progression_dict.keys())  # Add any other base titles as necessary
#scraped_titles = df_j_cleaned["displayTitle"].unique().tolist()

# Function to normalize job titles
def normalize_title(title):
    # Lowercase and remove non-alphanumeric characters, replace with spaces
    title = re.sub(r'[^a-z0-9]', ' ', title.lower())
    # Remove extra whitespace
    title = re.sub(r'\s+', ' ', title).strip()

        # Remove common prefixes/suffixes
    #title = re.sub(r'\b(senior|junior|associate|expert|technical|lead|l\d+)\b', '', title)
    title = re.sub(r'\b(sr|senior|expert|technical|lead|l\d+)\b', 'Senior', title)
    title = re.sub(r'\b(III|6|5|4|3|l\d+)\b', 'Senior', title)
    # Replace specific terms with standardized equivalents
    title = re.sub(r'\b(sr\.?|senior)\b', 'senior', title)  # Replace 'sr' or 'senior' with 'senior'
    title = re.sub(r'\b(jr\.?|junior)\b', 'junior', title) 

    # Convert to lower case and remove special characters
    title = re.sub(r'[^a-z\s]', '', title.lower())

    # Strip extra whitespace
    title = re.sub(r'\s+', ' ', title).strip()
    return title

# Create a set of base job titles from your job progression dictionary
base_job_titles = set()
for titles_list in sorted_full_job_progression_dict.values():
    for title in titles_list:
        base_job_titles.add(normalize_title(title))  # Add the normalized base title



# Function to find the best matching title from the base job titles
def match_title_to_base(scraped_title, base_job_titles):
    # Normalize the scraped job title
    normalized_title = normalize_title(scraped_title)
    # Check if the normalized title exactly matches one of the base job titles
    if normalized_title in base_job_titles:
        return normalized_title  # Return the matching base title
    
    # Partial match checking - longer base titles are checked first to match more specific job titles
    sorted_base_titles = sorted(base_job_titles, key=len, reverse=True)
    for base_title in sorted_base_titles:
        if base_title in normalized_title:
            if "senior" in normalized_title and "senior" not in base_title:
                return "senior "+base_title
            if "junior" in normalized_title and "junior" not in base_title:
                return "junior "+base_title
            return base_title
    
    return "unmatched"  # Return "unmatched" or some default value if no match is found

# Define a function to match job titles using fuzzy string matching
def fuzzy_match_title(scraped_title, base_job_titles, threshold=90):
    # Use the process function to find the closest match above a certain score threshold
    best_match, score = process.extractOne(scraped_title, base_job_titles)
    # Only accept the match if the score is above the threshold
    return best_match if score >= threshold else "unmatched"

'''
# Match each scraped title to a base title
matched_titles = {title: match_title_to_base(title, base_job_titles) for title in scraped_titles}
matched_titles_fuzzy = {}
for title in [normalize_title(each) for each in scraped_titles]:
    matched_titles_fuzzy[title] = fuzzy_match_title(title, base_job_titles)

def fill_best_value(matched_titles_fuzzy,matched_titles):
    for k,v in matched_titles_fuzzy:
        if v==None:
            matched_titles_fuzzy[k]= matched_titles[k]

    for k,v in matched_titles_fuzzy:
        if v==None:
            matched_titles_fuzzy[k]= matched_titles[k]
'''


df_j_cleaned["title"] = df_j_cleaned["title"].apply(lambda x: fuzzy_match_title(x,base_job_titles))
df_j_cleaned["title"] = df_j_cleaned["title"].apply(lambda x: match_title_to_base(x,base_job_titles))
df_j_cleaned["displayTitle"] = df_j_cleaned["displayTitle"].apply(lambda x: fuzzy_match_title(x,base_job_titles))
df_j_cleaned["displayTitle"] = df_j_cleaned["displayTitle"].apply(lambda x: match_title_to_base(x,base_job_titles))

df_j_cleaned["title"] = df_j_cleaned["title"].apply(lambda x: match_title_to_base(x,base_job_titles))





In [197]:
columns_most_important = ['company', 'truncatedCompany',
 'companyRating',
 'companyReviewCount',
 'displayTitle',
  'title',
 'normTitle',
 'min',
 'max',
 'type',
 'snippet',
 'jobDescription','skills_tagged']

df_j_cleaned = df_j_cleaned[columns_most_important]


In [225]:
#clean course data 
def clean_course_df(df):
    df["full_description"] = df["description"]+df["what_you_will_learn_data"]+df["objectives"]
    df = clean_lem_stop(df,"full_description")
    course_description = df["full_description"].tolist()
    return df,course_description

df_c_cleaned,course_descriptions = clean_course_df(df_c)

df_c_cleaned["skills_tagged"] = df_c_cleaned["full_description"].apply(lambda x:phrase_matcher_model(x,skills_list))


df_c_cleaned.columns.tolist()

course_columns_imp = [
 'title',
 'url',
 'price_detail',
 'description',
 'headline',
 'num_subscribers',
 'rating',
 'num_reviews',
 'num_quizzes',
 'num_lectures',
 'num_curriculum_items',
 'requirements_data',
 'what_you_will_learn_data',
 'labels',
 'target_audiences',
 'estimated_content_length',
 'content_info',
 'instructional_level',
 'objectives',
 'full_description',
 'skills_tagged']

df_c_cleaned = df_c_cleaned[course_columns_imp]


def  course_price_detail(x):
    cou_dict = ast.literal_eval(x)
    return cou_dict["amount"]

def  category_primary(x):
    cou_dict = ast.literal_eval(x)
    return cou_dict["url"]

def  labels_return_title(x):
    cou_dict = ast.literal_eval(x)
    return cou_dict[0]["title"]


def  labels_return_url(x):
    cou_dict = ast.literal_eval(x)
    return cou_dict[0]["url"]


def  labels_return_display_name(x):
    cou_dict = ast.literal_eval(x)
    return cou_dict[0]["display_name"]

df_c_cleaned["price"] = df_c_cleaned["price_detail"].apply(lambda x: course_price_detail(x))
df_c_cleaned["requirements_data"] = df_c_cleaned["requirements_data"].apply(lambda x:ast.literal_eval(x)[0])
df_c_cleaned["course_title"]=df_c_cleaned["labels"].apply(lambda x:labels_return_title(x))
df_c_cleaned["course_url"] =df_c_cleaned["labels"].apply(lambda x:labels_return_url(x))
df_c_cleaned["course_display_name"] =df_c_cleaned["labels"].apply(lambda x:labels_return_display_name(x))
df_c_cleaned["objectives"] =df_c_cleaned["objectives"].apply(lambda x:ast.literal_eval(x)[0])
del df_c_cleaned["labels"]
df_c_cleaned["what_you_will_learn_data"] = df_c_cleaned["what_you_will_learn_data"].apply(lambda x:ast.literal_eval(x)[0])
df_c_cleaned["target_audiences"] = df_c_cleaned["target_audiences"].apply(lambda x:ast.literal_eval(x)[0])
del df_c_cleaned["price_detail"]
clean_course_columns =["title",
"description",
"headline",
"requirements_data",
"what_you_will_learn_data",
"target_audiences",
"objectives",
"full_description",
"course_title",
"course_display_name"]
for clean_column in clean_course_columns:
    df_c_cleaned = clean_lem_stop(df_c_cleaned,clean_column)




In [186]:
xs = set()
for i,each in enumerate(df_c_cleaned["skills_tagged"]):
    if len(each)==0:
        print(i)
    for each_v in each:
        xs.add(each_v)

In [187]:
df_c_cleaned[2:4]

,title,url,price_detail,description,headline,num_subscribers,rating,num_reviews,num_quizzes,num_lectures,num_curriculum_items,requirements_data,what_you_will_learn_data,target_audiences,estimated_content_length,content_info,instructional_level,objectives,full_description,skills_tagged,price,course_title,course_url,course_display_name
2,data science course complete data science boot...,https://www.udemy.com/course/the-data-science-...,"{'amount': 109.99, 'currency': 'USD', 'price_s...",p strong problem strong p p data scientist bes...,complete data science training mathematics sta...,634583,4.594564,131509,281,518,799,prior experience required start basics,course provides entire toolbox need data scien...,course want data scientist want learn field,1906,32 total hours,All Levels,course provides entire toolbox need data scien...,p strong problem strong p p data scientist bes...,"{python programming language, statistical anal...",109.99,data science,https://www.udemy.com/topic/data-science/,data science
3,r programming z r data science real exercises,https://www.udemy.com/course/r-programming/,"{'amount': 94.99, 'currency': 'USD', 'price_st...",p learn r programming p p lots r courses lectu...,learn programming r r studio data analytics da...,261248,4.662838,51993,5,89,94,prior knowledge experience needed passion succ...,learn program r good level,course want learn program r,638,10.5 total hours,All Levels,learn program r good level,p learn r programming p p lots r courses lectu...,{statistical},94.99,data mining,https://www.udemy.com/topic/data-mining/,data mining


In [198]:
df_j_cleaned[2:4]

,company,truncatedCompany,companyRating,companyReviewCount,displayTitle,title,normTitle,min,max,type,snippet,jobDescription,skills_tagged
2,"CGI Group, Inc.","CGI Group, Inc.",3.6,3570.0,data engineer,data engineer,Technical Lead,NaN,NaN,NaN,Technical Lead - AWS Data Engineer (Python) - ...,technical lead aws data engineer python remote...,"{computer science, bitbucket, c, big data, dev..."
3,Walmart Advanced Systems & Robotics,Walmart Advanced Systems & Robotics,NaN,NaN,data engineer,data engineer,Data Engineer,NaN,NaN,NaN,Walmart Advanced Systems &amp; Robotics is a f...,walmart advanced system amp robotics fast grow...,"{data integration, computer science, test data..."


In [199]:
sorted_full_job_progression_dict

{'AI Architect': ['AI Project Manager', 'Head of AI', 'Chief AI Officer'],
 'AI Developer': ['Senior AI Developer',
  'AI Architect',
  'AI Project Manager',
  'Head of AI',
  'Chief AI Officer'],
 'AI Project Manager': ['Head of AI', 'Chief AI Officer'],
 'Big Data Architect': ['Head of Big Data',
  'Director of Data Engineering',
  'Chief Information Officer (CIO)'],
 'Big Data Engineer': ['Senior Big Data Engineer',
  'Big Data Architect',
  'Head of Big Data',
  'Director of Data Engineering',
  'Chief Information Officer (CIO)'],
 'Business Analysis Manager': ['Business Intelligence Analyst',
  'Director of Business Analysis',
  'Chief Strategy Officer',
  'Data Analyst',
  'Data Scientist'],
 'Business Analyst': ['Senior Business Analyst',
  'Business Analysis Manager',
  'Business Intelligence Analyst',
  'Director of Business Analysis',
  'Chief Strategy Officer',
  'Data Analyst',
  'Data Scientist'],
 'Business Intelligence Analyst': ['Senior Business Intelligence Analyst',
 

In [210]:
# Generate synthetic user data
def generate_synthetic_data(df_j_cleaned, df_c_cleaned, skills_df, num_samples=1000):
    synthetic_data = []
    for _ in range(num_samples):
        current_job, current_skills = "Data Engineer", df_j_cleaned[df_j_cleaned["title"]=="data engineer"]["skills_tagged"].values[0]
        # Select a job to progress to that requires at least one additional skill
        next_job, next_skills = "Senior Data Engineer", df_j_cleaned[df_j_cleaned["title"]=="senior data engineer"]["skills_tagged"].values[0]
        '''
        while set(next_skills).issubset(current_skills):
            next_job, next_skills = select_random_job(jobs_df)
        '''
        # Find the skill gap
        skill_gap = list(set(next_skills) - set(current_skills))
        
        # Find a course that teaches one of the skills in the gap
        course = select_relevant_course(courses_df, skill_gap)
        if course is not None:
            course_name = course.Course.values[0]
            course_skills = course.Skills_Taught.values[0]
        else:
            # If no single course covers the gap, select a random one for this synthetic example
            course_name = "Random Course"
            course_skills = "Random Skill"
        
        # Append to the synthetic data list
        synthetic_data.append({
            'Current Job': current_job,
            'Next Job': next_job,
            'Current Skills': current_skills,
            'Next Skills': next_skills,
            'Skill Gap': skill_gap,
            'Course Taken': course_name,
            'Course Skills': course_skills
        })
    
    return pd.DataFrame(synthetic_data)

{'job_path': ['Data Engineer', 'Senior Data Engineer', 'Senior Data Engineer'], 'courses_taken': [], 'skills': set()}


In [215]:
df_j_cleaned[df_j_cleaned["title"]=="data engineer"]["skills_tagged"].values[0]

{'analytics',
 'c',
 'data engineering',
 'data infrastructure',
 'data manipulation',
 'data modeling',
 'data visualization',
 'predictive modeling',
 'problem solving',
 'python',
 'sql',
 'statistical analysis'}

In [217]:
current_job, current_skills = "Data Engineer", df_j_cleaned[df_j_cleaned["title"]=="data engineer"]["skills_tagged"].values[0]
# Select a job to progress to that requires at least one additional skill
next_job, next_skills = "Senior Data Engineer", df_j_cleaned[df_j_cleaned["title"]=="senior data engineer"]["skills_tagged"].values[0]
'''
while set(next_skills).issubset(current_skills):
    next_job, next_skills = select_random_job(jobs_df)
'''
# Find the skill gap
skill_gap = list(set(next_skills) - set(current_skills))
skill_gap

['cloud infrastructure',
 'computer science',
 'devops',
 'data governance',
 'data acquisition',
 'java',
 'dataset',
 'ci cd']

In [219]:
def select_relevant_course(df_c_cleaned, required_skills):
    relevant_courses = df_c_cleaned[df_c_cleaned['skills_tagged'].apply(lambda x: any(skill in x for skill in required_skills))]
    if not relevant_courses.empty:
        return relevant_courses.sample()
    return None

select_relevant_course(df_c_cleaned,skill_gap)

In [220]:
import pandas as pd

def calculate_course_score(course, skill_gap, weight_skill_match=1, weight_subscribers=0.001, weight_rating=0.5):
    # Calculate the proportion of skill gap covered by course
    skills_matched = sum(skill in course['skills_tagged'] for skill in skill_gap)
    skill_match_score = skills_matched / len(skill_gap) if skill_gap else 0

    # Factor in course popularity and rating
    subscriber_score = course['num_subscribers'] * weight_subscribers if 'num_subscribers' in course else 0
    rating_score = course['rating'] * weight_rating if 'rating' in course else 0

    # Calculate total score
    total_score = (skill_match_score * weight_skill_match) + subscriber_score + rating_score
    return total_score

def select_relevant_course(df_c_cleaned, skill_gap):
    # Apply the scoring function to each course
    df_c_cleaned['course_score'] = df_c_cleaned.apply(calculate_course_score, axis=1, skill_gap=skill_gap)

    # Sort courses based on the calculated score
    df_c_cleaned = df_c_cleaned.sort_values(by='course_score', ascending=False)

    # Select the top course if available
    top_course = df_c_cleaned.iloc[0] if not df_c_cleaned.empty else None
    return top_course

# Assuming df_c_cleaned is your cleaned courses dataframe and skill_gap is a list of skills that user lacks
df_c_cleaned = pd.DataFrame({
    # your courses dataframe content
})


# Select the most relevant course based on the skill gap
relevant_course = select_relevant_course(df_c_cleaned, skill_gap)


ValueError: Cannot set a DataFrame with multiple columns to the single column course_score

In [221]:
df_c_cleaned.apply(calculate_course_score, axis=1, skill_gap=skill_gap)

""


In [226]:
skill_gap

skills_matched = sum(skill in df_c_cleaned['skills_tagged'] for skill in skill_gap)
skill_match_score = skills_matched / len(skill_gap) if skill_gap else 0




In [228]:
skill_match_score

0.0

In [244]:
import pandas as pd

def calculate_course_overlap(course_skills, skill_gap):
    # Calculate the number of skills in the gap that the course covers
    overlap = len(course_skills.intersection(skill_gap))
    # Calculate the proportion of the skill gap covered
    if skill_gap:
        coverage = overlap / len(skill_gap)
    else:
        coverage = 0
    return overlap, coverage

df_c_cleaned['overlap'], df_c_cleaned['coverage'] = zip(*df_c_cleaned['skills_tagged'].apply(calculate_course_overlap, skill_gap=set(skill_gap)))

In [235]:
df_c_cleaned["overlap"]

0     0
1     0
2     0
3     0
4     1
     ..
35    0
36    0
37    0
38    0
39    0
Name: overlap, Length: 100, dtype: int64

In [242]:
def select_relevant_course(df_c_cleaned, skill_gap, weight_coverage=1, weight_subscribers=0.001, weight_rating=0.5):
    # Add a new column for the overlap score
    df_c_cleaned['overlap'], df_c_cleaned['coverage'] = zip(*df_c_cleaned['skills_tagged'].apply(
        calculate_course_overlap, skill_gap=set(skill_gap)))


    # Find the best course by sorting by course score
    best_course = df_c_cleaned.sort_values(by='course_score', ascending=False).head(1)

    return best_course
    
# Select the most relevant course for a given skill gap
best_course = select_relevant_course(df_c_cleaned, skill_gap)

best_course

In [259]:
df_c_cleaned["title"].unique()

array(['machine learning z ai python r chatgpt bonus',
       'python data science machine learning bootcamp',
       'data science course complete data science bootcamp',
       'r programming z r data science real exercises',
       'deep learning z neural networks ai chatgpt bonus',
       'statistics data science business analysis',
       'data science z hands exercises chatgpt bonus',
       'artificial intelligence z build ai incl chatgpt',
       'machine learning data science generative ai python',
       'python z python data science real exercises',
       'spark python big data pyspark', 'data analysis pandas python',
       'complete machine learning data science bootcamp',
       'apache spark scala hands big data',
       'complete guide tensorflow deep learning python',
       'data science machine learning bootcamp r',
       'taming big data apache spark python hands',
       'nlp natural language processing python',
       'introduction machine learning data science'

In [243]:
best_course

,title,url,description,headline,num_subscribers,rating,num_reviews,num_quizzes,num_lectures,num_curriculum_items,requirements_data,what_you_will_learn_data,target_audiences,estimated_content_length,content_info,instructional_level,objectives,full_description,skills_tagged,price,course_title,course_url,course_display_name,overlap,coverage,course_score
0,machine learning z ai python r chatgpt bonus,https://www.udemy.com/course/machinelearning/,p interested field machine learning nbsp cours...,learn create machine learning algorithms pytho...,993456,4.513425,176950,39,468,507,high school mathematics level,master machine learning python r,interested machine learning,2559,42.5 total hours,All Levels,master machine learning python r,p interested field machine learning nbsp cours...,"{multiple linear regression, data science, dee...",109.99,data science,https://www.udemy.com/topic/data-science/,data science,0,0.0,995.712713


In [250]:
for k,row in df_c_cleaned[df_c_cleaned["overlap"]==1].iterrows():
    print(k,row["skills_tagged"])

4 {'deep learning', 'algorithms', 'deep learning methods', 'python', 'analytics', 'autoencoders', 'dataset', 'probability', 'artificial intelligence', 'amazon', 'convolutional neural networks', 'artificial neural networks'}
13 {'big data', 'apache', 'mapreduce', 'cloud computing', 'apache spark', 'sql', 'dataset', 'data analysis', 'amazon', 'python'}
18 {'computer science', 'data science', 'algorithms', 'big data', 'machine learning algorithms', 'data analysis', 'artificial intelligence', 'python', 'data mining'}
40 {'deep learning', 'data science', 'algorithms', 'python', 'machine learning algorithms', 'resnet', 'dataset', 'convolutional neural networks'}
43 {'deep learning', 'data science', 'sentiment analysis', 'algorithms', 'data preprocessing', 'dataset', 'probability', 'python'}
50 {'jupyter notebook', 'airflow', 'continuous integration', 'ci cd', 'python'}
4 {'deep learning', 'algorithms', 'deep learning methods', 'python', 'analytics', 'autoencoders', 'dataset', 'probability', 

In [251]:
skill_gap

['cloud infrastructure',
 'computer science',
 'devops',
 'data governance',
 'data acquisition',
 'java',
 'dataset',
 'ci cd']

In [256]:
df_c_cleaned

,title,url,description,headline,num_subscribers,rating,num_reviews,num_quizzes,num_lectures,num_curriculum_items,requirements_data,what_you_will_learn_data,target_audiences,estimated_content_length,content_info,instructional_level,objectives,full_description,skills_tagged,price,course_title,course_url,course_display_name,overlap,coverage,course_score
0,machine learning z ai python r chatgpt bonus,https://www.udemy.com/course/machinelearning/,p interested field machine learning nbsp cours...,learn create machine learning algorithms pytho...,993456,4.513425,176950,39,468,507,high school mathematics level,master machine learning python r,interested machine learning,2559,42.5 total hours,All Levels,master machine learning python r,p interested field machine learning nbsp cours...,"{multiple linear regression, data science, dee...",109.99,data science,https://www.udemy.com/topic/data-science/,data science,0,0.000,995.712713
1,python data science machine learning bootcamp,https://www.udemy.com/course/python-for-data-s...,p ready start path data scientist p p comprehe...,learn use numpy pandas seaborn matplotlib plot...,674125,4.588927,136746,1,184,185,programming experience,use python data science machine learning,course meant people programming experience,1494,25 total hours,All Levels,use python data science machine learning,p ready start path data scientist p p comprehe...,"{data science, deep learning, big data, machin...",99.99,data science,https://www.udemy.com/topic/data-science/,data science,0,0.000,676.419464
2,data science course complete data science boot...,https://www.udemy.com/course/the-data-science-...,p strong problem strong p p data scientist bes...,complete data science training mathematics sta...,634583,4.594564,131509,281,518,799,prior experience required start basics,course provides entire toolbox need data scien...,course want data scientist want learn field,1906,32 total hours,All Levels,course provides entire toolbox need data scien...,p strong problem strong p p data scientist bes...,"{python programming language, statistical anal...",109.99,data science,https://www.udemy.com/topic/data-science/,data science,0,0.000,636.880282
3,r programming z r data science real exercises,https://www.udemy.com/course/r-programming/,p learn r programming p p lots r courses lectu...,learn programming r r studio data analytics da...,261248,4.662838,51993,5,89,94,prior knowledge experience needed passion succ...,learn program r good level,course want learn program r,638,10.5 total hours,All Levels,learn program r good level,p learn r programming p p lots r courses lectu...,{statistical},94.99,data mining,https://www.udemy.com/topic/data-mining/,data mining,0,0.000,263.579419
4,deep learning z neural networks ai chatgpt bonus,https://www.udemy.com/course/deeplearning/,p seen kickstarter p p artificial intelligence...,learn create deep learning algorithms python m...,366900,4.501419,44403,0,230,230,high school mathematics level,understand intuition artificial neural networks,interested deep learning,1367,23 total hours,All Levels,understand intuition artificial neural networks,p seen kickstarter p p artificial intelligence...,"{deep learning, algorithms, deep learning meth...",99.99,deep learning,https://www.udemy.com/topic/deep-learning/,deep learning,1,0.125,369.275710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,deep learning computer vision z ai chatgpt bon...,https://www.udemy.com/course/computer-vision-a-z/,p definitely heard ai deep learning ask positi...,wizard latest computer vision tools exist dete...,47716,4.196091,6515,6,95,101,high school maths,toolbox powerful computer vision models,interested computer vision artificial intellig...,668,11 total hours,All Levels,toolbox powerful computer vision models,p definitely heard ai deep learning ask positi...,"{artificial intelligence, deep learning}",84.99,artificial intelligence,https://www.udemy.com/topic/

In [257]:
df_c_cleaned[df_c_cleaned["overlap"]==1]

,title,url,description,headline,num_subscribers,rating,num_reviews,num_quizzes,num_lectures,num_curriculum_items,requirements_data,what_you_will_learn_data,target_audiences,estimated_content_length,content_info,instructional_level,objectives,full_description,skills_tagged,price,course_title,course_url,course_display_name,overlap,coverage,course_score
4,deep learning z neural networks ai chatgpt bonus,https://www.udemy.com/course/deeplearning/,p seen kickstarter p p artificial intelligence...,learn create deep learning algorithms python m...,366900,4.501419,44403,0,230,230,high school mathematics level,understand intuition artificial neural networks,interested deep learning,1367,23 total hours,All Levels,understand intuition artificial neural networks,p seen kickstarter p p artificial intelligence...,"{deep learning, algorithms, deep learning meth...",99.99,deep learning,https://www.udemy.com/topic/deep-learning/,deep learning,1,0.125,369.275710
13,apache spark scala hands big data,https://www.udemy.com/course/apache-spark-with...,p strong new completely updated recorded spark...,apache spark tutorial hands examples analyzing...,94162,4.632853,17262,6,73,79,prior programming scripting experience require...,develop distributed code scala programming lan...,software engineers want expand skills world bi...,538,9 total hours,All Levels,develop distributed code scala programming lan...,p strong new completely updated recorded spark...,"{big data, apache, mapreduce, cloud computing,...",94.99,apache spark,https://www.udemy.com/topic/apache-spark/,apache spark,1,0.125,96.603427
18,introduction machine learning data science,https://www.udemy.com/course/machine-learning-...,p strong course recently updated nov strong p ...,primer machine learning data science revealed ...,62512,4.555354,13772,3,64,67,passion learn basic computer skills,genuinely understand computer science algorith...,load python start r need course introductory c...,333,5.5 total hours,Beginner Level,genuinely understand computer science algorith...,p strong course recently updated nov strong p ...,"{computer science, data science, algorithms, b...",84.99,data science,https://www.udemy.com/topic/data-science/,data science,1,0.125,64.914677
40,deep learning advanced computer vision gans ssd,https://www.udemy.com/course/advanced-computer...,p strong latest update strong instead ssd use ...,vgg resnet inception ssd retinanet neural styl...,36728,4.729925,5620,0,140,140,know build train use cnn library preferably py...,understand apply transfer learning,students professionals want knowledge computer...,941,15.5 total hours,Intermediate Level,understand apply transfer learning,p strong latest update strong instead ssd use ...,"{deep learning, data science, algorithms, pyth...",119.99,deep learning,https://www.udemy.com/topic/deep-learning/,deep learning,1,0.125,39.217962
43,deep learning convolutional neural networks py...,https://www.udemy.com/course/deep-learning-con...,p nbsp nbsp nbsp tensorflow python nbsp p p le...,tensorflow cnns computer vision natural langua...,37527,4.714412,5406,0,117,117,basic math taking derivatives matrix arithmeti...,understand convolution useful deep learning,students professionals interested deep learnin...,804,13.5 total hours,All Levels,understand convolution useful deep learning,p nbsp nbsp nbsp tensorflow python nbsp p p le...,"{deep learning, data science, sentiment analys...",109.99,deep learning,https://www.udemy.com/topic/deep-learning/,deep learning,1,0.125,40.009206
50,deployment machine learning models,https://www.udemy.com/course/deployment-of-mac...,p welcome deployment machine learning models s...,learn integrate robust reliable machine learni...,33224,4.372408,4992,1,219,224,python installation,build machine learning model apis deploy model...,data scientists want deploy machine learning m...,627,10.5 total hours,Intermediate Level,build machine learning model apis deploy model...,p welcome deployment machine learning models

In [246]:
skill_gap

['cloud infrastructure',
 'computer science',
 'devops',
 'data governance',
 'data acquisition',
 'java',
 'dataset',
 'ci cd']

In [203]:
sorted_full_job_progression_dict

{'AI Architect': ['AI Project Manager', 'Head of AI', 'Chief AI Officer'],
 'AI Developer': ['Senior AI Developer',
  'AI Architect',
  'AI Project Manager',
  'Head of AI',
  'Chief AI Officer'],
 'AI Project Manager': ['Head of AI', 'Chief AI Officer'],
 'Big Data Architect': ['Head of Big Data',
  'Director of Data Engineering',
  'Chief Information Officer (CIO)'],
 'Big Data Engineer': ['Senior Big Data Engineer',
  'Big Data Architect',
  'Head of Big Data',
  'Director of Data Engineering',
  'Chief Information Officer (CIO)'],
 'Business Analysis Manager': ['Business Intelligence Analyst',
  'Director of Business Analysis',
  'Chief Strategy Officer',
  'Data Analyst',
  'Data Scientist'],
 'Business Analyst': ['Senior Business Analyst',
  'Business Analysis Manager',
  'Business Intelligence Analyst',
  'Director of Business Analysis',
  'Chief Strategy Officer',
  'Data Analyst',
  'Data Scientist'],
 'Business Intelligence Analyst': ['Senior Business Intelligence Analyst',
 

In [201]:
df_c_cleaned['skills_tagged'] 

TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
# train a spacy model to identify skills 
# create entities 
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.util import filter_spans

nlp = spacy.load("en_core_web_sm")  # Load the model
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")  # Create the matcher object


# Convert the skills list to Doc objects and add them as patterns to the matcher
patterns = [nlp.make_doc(skill) for skill in skills_list]
matcher.add("Skills", patterns)


def get_entities(TRAIN_DATA):
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            print(ent)

TRAIN_DATA = []

# Loop through the job descriptions and create training data
for job_description in job_descriptions:
    # Process the job description to create a Spacy Doc
    doc = nlp(job_description)
    # Match the patterns to the doc
    matches = matcher(doc)
    # Create Span objects for the matched sequences
    spans = [Span(doc, start, end, label="SKILLS") for match_id, start, end in matches]
    # Filter the spans to remove overlaps
    #print(spans)
    filtered_spans = filter_spans(spans)
    #print(spans)
    entities = [(span.start_char, span.end_char, span) for span in filtered_spans]
    TRAIN_DATA.append((job_description, {"entities": entities}))

#print(TRAIN_DATA)





# train the model

#train the model 
starting_fresh = False
# Load a pre-existing spaCy model
nlp = spacy.load('en_core_web_sm')  # for example
import random
from spacy.util import minibatch
# Get the Named Entity Recognizer component in the pipeline
ner = nlp.get_pipe('ner')
from spacy.training import Example
from pathlib import Path

# Add new entity labels to 'ner'
for _, annotations in TRAIN_DATA:
    for start,end,label in annotations.get('entities'):
        #print(label)
        ner.add_label(str(label))


# Disable other pipes during training
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']


# Begin training
with nlp.disable_pipes(*unaffected_pipes):
    if starting_fresh:
        nlp.begin_training()

    for itn in range(5):
        random.shuffle(TRAIN_DATA)
        losses = {}

        # Batch up the examples using spaCy's minibatch
        for batch in minibatch(TRAIN_DATA, size=2):
            examples = []
            for text, annotations in batch:
                # Create a Spacy Doc from the text
                doc = nlp.make_doc(text)
                # Create an Example using the annotations
                example = Example.from_dict(doc, annotations)
                examples.append(example)

            # Update the model
            nlp.update(
                examples,
                drop=0.5,  # Dropout - make it harder to memorize data
                losses=losses
            )
        print(losses)

'''
from pathlib import Path
output_dir = Path('/Users/nyzy/nitzmali/job_transition_pathway/models/skills_tag_spacy_nlp_model')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)
'''


# Split your TRAIN_DATA into train, validate and test sets
def train_test_val_split(data, test_size=0.2, val_size=0.25, random_state=42):
    # Calculate actual validation set size of the remaining data after test split
    val_size_adjusted = val_size / (1 - test_size)
    # Split off test set from available data
    train_val_data, test_data = train_test_split(data, test_size=test_size, random_state=random_state)
    # Split remaining data into training and validation sets
    train_data, val_data = train_test_split(train_val_data, test_size=val_size_adjusted, random_state=random_state)
    return train_data, val_data, test_data
train_data, val_data, test_data = train_test_val_split(TRAIN_DATA, test_size=0.2, val_size=0.25)


#########################################################################################
# Load the model you want to evaluate
from spacy import displacy
nlp = spacy.load('/Users/nyzy/nitzmali/job_transition_pathway/models/skills_tag_spacy_nlp_model')  # replace with your model name

# Split your TRAIN_DATA into train, validate and test sets
train_data, val_data, test_data = train_test_val_split(TRAIN_DATA, test_size=0.2, val_size=0.25)

# Convert the validation data to spaCy's Example format
examples = []
for input_, annots in val_data:
    pred = nlp(input_)
    example = Example.from_dict(pred, annots)
    examples.append(example)


# Use the Scorer to score the examples
scorer = Scorer(nlp)
scores = scorer.score(examples)


precision = scores['ents_p']
recall = scores['ents_r']
f_score = scores['ents_f']

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F-score: {f_score}")

examples[0]


# Assume 'nlp' is your loaded NLP model
for text, annots in val_data[3:5]:  # Let's use val_data as an example
    doc = nlp(text)  # Process the text to predict entities

    '''
    print("Predictions by model:")
    for ent in doc.ents:
        print("Predictions")
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)

    # Now print the correct data for comparison
    print("\nCorrect labels:")
    for start, end, label in annots['entities']:
        print("Actual")
        #print(text[start:end], start, end, label)
    '''
    # You can use displacy here as well if you prefer visual comparison
    displacy.render(doc, style="ent", jupyter=True)

    # Adding a separation for readability between different examples
    print("\n" + "-" * 50 + "\n")


